In [ ]:
import warnings
warnings.simplefilter("ignore")
import pandas as pd
from sklearn.neighbors import KDTree
import tsfresh as ts 
import matplotlib.pyplot as plt 
from sklearn.linear_model import Ridge
import numpy as np
import pymc

In [ ]:
files = pd.read_excel('/home/velaraptor/Downloads/Raw Data 10yrs (2018).xlsx', header=1)

In [ ]:
# use KDTree, Regression for Prediction, and Forecast, and use tsfresh, and use bayesian 
files.head()

In [ ]:
files[files['FantPos'] == 'QB'].head()

In [ ]:
files.groupby('FantPos').size()

In [ ]:
# fig, ax = plt.subplots(figsize=(20,10))


# files[files['FantPos'] == 'QB'].groupby('Name').plot(x='Year', y='FantPt', ax=ax, legend=False, alpha=0.25)
# plt.show()

In [ ]:
# fig, ax = plt.subplots(figsize=(20,10))


# files[files['FantPos'] == 'RB'].groupby('Name').plot(x='Year', y='FantPt', ax=ax, legend=False, alpha=0.25)
# plt.show()

In [ ]:
# fig, ax = plt.subplots(figsize=(20,10))


# files[files['FantPos'] == 'TE'].groupby('Name').plot(x='Year', y='FantPt', ax=ax, legend=False, alpha=0.25)
# plt.show()

In [ ]:
# fig, ax = plt.subplots(figsize=(20,10))


# files[files['FantPos'] == 'WR'].groupby('Name').plot(x='Year', y='FantPt', ax=ax, legend=False, alpha=0.25)
# plt.show()

In [ ]:
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from sklearn.ensemble import AdaBoostRegressor
from tsfresh.utilities.dataframe_functions import impute

In [ ]:
files = files.fillna(0)
groups = files.groupby('Name')

import tqdm

forecast_df = []
for name, group in tqdm.tqdm(groups):
    if len(group) > 1:
        group.index = group.Year
        df_shift, y = make_forecasting_frame(group["FantPt"], kind=name, max_timeshift=10, rolling_direction=1)
        forecast_df.append(df_shift)

In [ ]:
f = files[files['Year'] != 2018]

In [ ]:
test_f = files[files['Year'] ==2018].Name.unique()

test_ff = files[files['Name'].isin(test_f)]

In [ ]:
from tsfresh.feature_extraction import extract_features, MinimalFCParameters, EfficientFCParameters

extracted_features = extract_features(f[['Year', 'Name', 'PPG', 'FantPt']], column_id="Name", column_sort="Year", default_fc_parameters=EfficientFCParameters())

In [ ]:
test_features = extract_features(test_ff[['Year', 'Name', 'PPG', 'FantPt']], column_id="Name", column_sort="Year", default_fc_parameters=EfficientFCParameters())

In [ ]:
ts_features = extracted_features.reset_index()
ts_features_test = test_features.reset_index()

In [ ]:
ts_features.columns

In [ ]:
ts_features = ts_features.dropna(axis=1)

In [ ]:
ts_features_test = ts_features_test[ts_features.columns]

In [ ]:
idx = files.groupby(['Name'])['Year'].transform(max) == files['Year']
y = files[idx][['Name', 'FantPt', 'Age']]

In [ ]:
ts_features = ts_features.merge(y, left_on='id',right_on='Name')

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
columns = np.append(ts_features.columns[2:len(ts_features.columns)-3].values, 'Age')
ada = AdaBoostRegressor(base_estimator=GradientBoostingRegressor(loss='quantile', max_depth=10), n_estimators=100, loss='exponential')
ada.fit(ts_features[columns], ts_features['FantPt'])

In [ ]:
ada.predict(ts_features[columns])

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
plt.scatter(ada.predict(ts_features[columns]), ts_features['FantPt'])
plt.show()

In [ ]:
# ts_features_test = ts_features_test.merge(y, left_on='id',right_on='Name')
ada.predict(ts_features_test[columns])

In [ ]:
pd.DataFrame([ada.predict(to_predict[columns]), to_predict['FantPt'], to_predict['Name']]).T

In [ ]:
ts_features

In [ ]:
#%%capture
from tsfresh.feature_extraction import extract_features, MinimalFCParameters, EfficientFCParameters
features_df = []
for sample in forecast_df[:2]:
    X = extract_features(sample, column_id="id", column_sort="time", column_value="value", impute_function=impute,
                     show_warnings=False, disable_progressbar=True, default_fc_parameters=MinimalFCParameters())
    X = X.reset_index()
    X.loc[:, 'Name'] = sample['kind']
    features_df.append(X)

In [ ]:
pd.concat(features_df)

In [ ]:
features_df[0].reset_index()

In [ ]:
files[files['Name']=='Adam Jennings']

In [ ]:
# get year in nfl, age, pos. fanpt, ppg, Yds, Yds.2, TD, TD.2, coach

In [ ]:
files.columns